In [24]:
# Library
import nltk
from nltk import grammar

In [2]:
>>> groucho_grammar = nltk.CFG.fromstring("""
... S -> NP VP
... PP -> P NP
... NP -> Det N | Det N PP | 'I'
... VP -> V NP | VP PP
... Det -> 'an' | 'my'
... N -> 'elephant' | 'pajamas'
... V -> 'shot'
... P -> 'in'
... """)

In [3]:
groucho_grammar

<Grammar with 13 productions>

In [4]:
sent = 'I shot an elephant in my pajamas'.split(' ')

In [5]:
parser = nltk.ChartParser(groucho_grammar)
trees = parser.parse(sent)
for tree in trees:
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [6]:
grammer1 = nltk.CFG.fromstring('''
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"''')

In [10]:
sent = 'Mary saw Bob'.split(' ')
rd_parser = nltk.RecursiveDescentParser(grammer1)

In [11]:
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


In [16]:
# grammer1 = nltk.data.load('file:mygrammar.cfg')

In [17]:
nltk.download('home')

[nltk_data] Error loading home: Package 'home' not found in index


False

In [18]:
sent = 'Mary saw a dog'.split()
for t in rd_parser.parse(sent):
    print(t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [19]:
'''
aadasdad 1100 000111 1 00 11001100 00001  11111 111 111 010 00 0000 
\b1*(01*0)*\b

1111
1010
1011110111011110
0101
00
000
00011
0001110
11111111111100
111111111111111100001111111
1
1
11
1111
010101
111
0011
111111
001001011


\b
\b
(1*01*?01*?)*|
\b(00)+\b|
\b(11)+\b)
\b



(?x)
\b
(?:(               #if
(?=\b(1*?01*?01*?)+)\b # condition check for the 0 is even
(?:                #true than if
(?=\b(0*?10*?10*?)+)\b # condition check for the 1 is even
\b(1*?01*?01*?)*\b|
#\b(00)+\b
)
)
|
\b(11)+|(00)+\b
)
\b
'''

'\naadasdad 1100 000111 1 00 11001100 00001  11111 111 111 010 00 0000 \n\x081*(01*0)*\x08\n\n1111\n1010\n1011110111011110\n0101\n00\n000\n00011\n0001110\n11111111111100\n111111111111111100001111111\n1\n1\n11\n1111\n010101\n111\n0011\n111111\n001001011\n\n\n\x08\n\x08\n(1*01*?01*?)*|\n\x08(00)+\x08|\n\x08(11)+\x08)\n\x08\n\n\n\n(?x)\n\x08\n(?:(               #if\n(?=\x08(1*?01*?01*?)+)\x08 # condition check for the 0 is even\n(?:                #true than if\n(?=\x08(0*?10*?10*?)+)\x08 # condition check for the 1 is even\n\x08(1*?01*?01*?)*\x08|\n#\x08(00)+\x08\n)\n)\n|\n\x08(11)+|(00)+\x08\n)\n\x08\n'

In [22]:
numtokens = 2
wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
wfst

[[None, None, None], [None, None, None], [None, None, None]]

In [23]:
def init_wfst(tokens,grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
        
    return wfst

In [33]:
def complete_wfst(wfst,tokens,grammar,trace = False):
    index = dict((p.rhs(),p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    
    for span in range(2,numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            
            for mid in range(start+1, end):
                nt1,nt2 = wfst[start][mid],wfst[mid][end]
                
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    
                    if trace:
                        print('[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]' % \
                             (start,nt1,mid,nt2,end,start,index[(nt1,nt2)],end))
    return wfst

In [46]:
def display(wfst,tokens):
    
    print('\nWFST ' + ' '.join([('%-4d'%i)  for i in range(1,len(wfst))]))
    
    for i in range(len(wfst)-1):
        print('%d    '%i,end='')
        for j in range(1,len(wfst)):
            print('%-5s'%(wfst[i][j] or '.'),end='')
        print()

In [47]:
tokens = 'I shot an elephant in my pajamas'.split()
wfst0 = init_wfst(tokens ,groucho_grammar)

In [48]:
display(wfst0,tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


In [49]:
wfst1 = complete_wfst(wfst0,tokens,groucho_grammar)

In [50]:
display(wfst1,tokens)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


In [51]:
wfst1 = complete_wfst(wfst0,tokens,groucho_grammar,trace = True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
